In [2]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.0 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=fc2c7f250d17c11017bae27d7d5a5c8eada9e0798695a4d91416ad1716b356db
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.7 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_US         dataframe_result_csv
 a0529.csv	        dataframe_result_csv_UK
 a0530.csv	        Github
 a0531.csv	        heatmap529_UK.ipynb
 a0601.csv	        heatmap530.ipynb
 a0602.csv	        Heat_Map_Online_language.ipynb
 a0603.csv	        __MACOSX
 csv		        non_violant1.csv
 csv.zip	        non_violant2.csv
 dataframe0603_UK.csv   UK
 dataframe_csv	        violant.csv
 dataframe_csv_UK      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_531 = 'UK/06_04.csv'

In [8]:
ct = pd.read_csv(ct_path_531)

In [9]:
ct.shape[1]

47

In [10]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,2054592981326565_2919503204835534,Facebook,2020-06-04 23:59:57,2021-04-24 15:46:24,youtube,Cornel West: The Future Of America Depends On ...,youtube.com,"Harvard University professor, author, and thin...",*** TAKE A GOOD LISTEN,1.0,https://www.youtube.com/watch?v=BkYJeMwlsto,https://www.facebook.com/groups/20545929813265...,2814,-30.000000,2.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,1,3,1,1,0,1,10176145,Bulgarians in USA United | Обединени Българи в...,921thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,Tulsa's Party Station and home of Elvis Duran ...,False,2009-03-03 23:35:31,en,0,10176145|2919503204835534
1,122310264455311_3256911610995145,Facebook,2020-06-04 23:59:57,2021-06-16 09:58:10,link,"T.I., Kevin Hart, Tiffany Haddish & More Atten...",wild104.iheart.com,The first of George Floyd's three public memor...,The first of George Floyd's three public memor...,1.0,http://ihe.art/yj2ad6P,https://www.facebook.com/122310264455311/posts...,140231,-10.000000,1.0,3,0,0,0,0,0,0,0,0,0,3,3,4,2,4,3,7,4,0,1,39988,WILD 104,wild104,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The Rio Grande Valley's Party Station for Hip...,True,2010-05-11 19:29:05,en,0,39988|3256911610995145
2,1250734815048111_3092238447564396,Facebook,2020-06-04 23:59:56,2021-08-16 19:32:24,link,Al Sharpton Attacks Trump At George Floyd’s Fu...,chicagodailybrief.tumblr.com,Today at the funeral for George Floyd Reverend...,What a disgrace…,1.0,https://chicagodailybrief.tumblr.com/post/6200...,https://www.facebook.com/groups/12507348150481...,28675,58.192308,1.0,44,120,588,2,9,17,60,673,0,0,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,wild104,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,The Rio Grande Valley's Party Station for Hip...,False,2010-05-11 19:29:05,en,0,7838630|3092238447564396
3,217290511618934_3551441844870434,Facebook,2020-06-04 23:59:56,2021-10-15 05:19:58,link,Reclaim the Block: fund our broader movement,docs.google.com,"To all our supporters, We are so grateful for ...",>READ ALL OF THIS BEFORE SEARCHING< Me and my ...,5.0,https://docs.google.com/document/d/1yLWGTQIe39...,https://www.facebook.com/217290511618934/posts...,37773,1.631579,1.0,23,0,1,7,0,0,0,0,0,0,8,0,1,3,0,0,0,0,0,0,9119246,Brad Kavanagh,officialbradkavanagh,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,Music maker and one quarter of the band FLYNT.,False,2011-04-13 22:38:32,en,0,9119246|3551441844870434
4,79146833756_10157710305038757,Facebook,2020-06-04 23:59:54,2021-08-31 17:54:38,native_video,Reclaim the Block: fund our broader movement,docs.google.com,"To all our supporters, We are so grateful for ...",Protesters peaceully march near Water and Broa...,1.0,https://www.facebook.com/tmj4/videos/186296016...,https://www.facebook.com/79146833756/posts/101...,348655,3.456026,2.0,357,47,515,118,2,7,4,11,0,19,162,38,27,46,11,11,6,6,0,4,25507,TMJ4 News,tmj4,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,TMJ4 News brings you southeast Wisconsin break...,True,2009-05-13 13:33:43,en,0,25507|10157710305038757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{'America': 1}
1,{'Minneapolis': 1},{'Minneapolis': 1}
2,{},{}
3,{},{'Minneapolis': 2}
4,{},{'Minneapolis': 2}


In [24]:
ct.to_csv('a0604.csv')

In [25]:
ct=pd.read_csv('a0604.csv')

In [26]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [27]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [28]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{'America': 1},{}
1,{'Minneapolis': 1},{'Minneapolis': 1},{}
3,{},{'Minneapolis': 2},{}
4,{},{'Minneapolis': 2},{}
5,{'Minneapolis': 1},{'Minneapolis': 1},{}


In [29]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
ct_ner['NER-msg']

0                   []
1        [Minneapolis]
3                   []
4                   []
5        [Minneapolis]
             ...      
44683           [U.S.]
44684    [Minneapolis]
44685    [Minneapolis]
44689               []
44690               []
Name: NER-msg, Length: 30459, dtype: object

In [31]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ct_ner['NER']

0                         [America]
1        [Minneapolis, Minneapolis]
3                     [Minneapolis]
4                     [Minneapolis]
5        [Minneapolis, Minneapolis]
                    ...            
44683                  [U.S., U.S.]
44684                 [Minneapolis]
44685                 [Minneapolis]
44689                   [Hollywood]
44690                    [Illinois]
Name: NER, Length: 30459, dtype: object

In [33]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ner_lst = ct_ner['NER'].values

In [36]:
ner_lst

array([list(['america']), list(['minneapolis']), list(['minneapolis']),
       ..., list(['minneapolis']), list(['hollywood']),
       list(['illinois'])], dtype=object)

In [37]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [38]:
list

['america',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'chicago',
 'illinois',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'iowa',
 'des moines',
 'minneapolis',
 'the united states',
 'minneapolis',
 'america',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'guam',
 'the united states',
 'guam',
 'minneapolis',
 'the united states',
 'guam',
 'brooklyn',
 'the united states',
 'pennsylvania ave.',
 'lafayette park',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'florida',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'minneapolis',
 'america'

In [39]:
c = collections.Counter()

In [40]:
dict_a=collections.Counter(list).most_common()

In [41]:
dict_a

[('minneapolis', 12691),
 ('america', 3487),
 ('us', 3078),
 ('minnesota', 2465),
 ('the united states', 1837),
 ('u.s.', 1808),
 ('washington', 942),
 ('los angeles', 783),
 ('new york', 683),
 ('houston', 676),
 ('chicago', 613),
 ('texas', 563),
 ('london', 516),
 ('usa', 474),
 ('uk', 457),
 ('florida', 456),
 ('new york city', 383),
 ('sussex', 348),
 ('california', 325),
 ('brooklyn', 324),
 ('nyc', 272),
 ('nebraska', 269),
 ('atlanta', 268),
 ('d.c.', 263),
 ('dallas', 259),
 ('hollywood', 247),
 ('north carolina', 235),
 ('denver', 225),
 ('philadelphia', 220),
 ('georgia', 206),
 ('virginia', 203),
 ('china', 203),
 ('seattle', 198),
 ('lincoln', 198),
 ('portland', 196),
 ('las vegas', 193),
 ('ohio', 186),
 ('richmond', 183),
 ('australia', 179),
 ('paris', 177),
 ('canada', 176),
 ('louisville', 176),
 ('united states', 173),
 ('|', 169),
 ('boston', 156),
 ('new orleans', 154),
 ('britain', 148),
 ('washington dc', 146),
 ('philando castile', 144),
 ('the united states of

In [42]:
dataframe531=pd.DataFrame(dict_a)

In [43]:
len(dict_a)

4823

In [44]:
dataframe531.to_csv('dataframe0604_UK.csv')